In [ ]:
# 步骤 1：加载数据集

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 数据预处理：将PIL图像转换为张量，并归一化
transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为张量（0-1）
    transforms.Normalize((0.1307,), (0.3081,))  # 标准化（MNIST的均值和标准差）
])

# 加载训练集和测试集
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

# 数据加载器（批处理、打乱、多线程）
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# 步骤 2：定义神经网络模型

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # 定义网络层
        self.fc1 = nn.Linear(784, 256)  # 输入层（28*28=784）→ 隐藏层1
        self.fc2 = nn.Linear(256, 128)  # 隐藏层1 → 隐藏层2
        self.fc3 = nn.Linear(128, 10)   # 隐藏层2 → 输出层（10个数字类别）
        self.relu = nn.ReLU()  # 激活函数

    def forward(self, x):
        # 定义前向传播
        x = x.view(-1, 784)  # 展平张量：(batch_size, 1, 28, 28) → (batch_size, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # 输出层无需激活，后续用CrossEntropyLoss计算损失
        return x

# 初始化模型
model = SimpleNN()
# 移到GPU（如果可用）
if torch.cuda.is_available():
    model = model.to("cuda")

In [ ]:
# 步骤 3：定义损失函数和优化器

# 损失函数：交叉熵损失（适用于分类任务）
criterion = nn.CrossEntropyLoss()
# 优化器：Adam
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()  # 切换到训练模式
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            # 移到GPU
            if torch.cuda.is_available():
                data, target = data.to("cuda"), target.to("cuda")
            
            # 清零梯度
            optimizer.zero_grad()
            # 前向传播
            output = model(data)
            # 计算损失
            loss = criterion(output, target)
            # 反向传播
            loss.backward()
            # 更新参数
            optimizer.step()

            running_loss += loss.item()
            # 每100批打印一次
            if batch_idx % 100 == 99:
                print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {running_loss/100:.4f}")
                running_loss = 0.0

# 训练5个epoch
train(model, train_loader, criterion, optimizer, epochs=5)

In [ ]:
# 步骤 5：测试模型

def test(model, test_loader):
    model.eval()  # 切换到评估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 禁用梯度计算，节省内存和计算资源
        for data, target in test_loader:
            if torch.cuda.is_available():
                data, target = data.to("cuda"), target.to("cuda")
            output = model(data)
            # 获取预测结果（概率最大的类别）
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f"测试集准确率: {100 * correct / total:.2f}%")

# 测试模型
test(model, test_loader)

In [ ]:
# 使用预训练模型（图像分类）
# PyTorch 的torchvision提供了大量预训练模型（如 ResNet、VGG、MobileNet），可直接用于迁移学习或推理。

from torchvision import models
from PIL import Image

# 加载预训练的ResNet50模型
model = models.resnet50(pretrained=True)
model.eval()  # 评估模式

# 图像预处理
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载并预处理图像
image = Image.open("cat.jpg")  # 替换为你的图像路径
input_tensor = transform(image).unsqueeze(0)  # 添加batch维度

# 推理
with torch.no_grad():
    output = model(input_tensor)

# 解析预测结果（ImageNet类别）
_, predicted_idx = torch.max(output, 1)
# 加载ImageNet类别标签
with open("imagenet_classes.txt") as f:
    classes = [line.strip() for line in f.readlines()]
print(f"预测类别: {classes[predicted_idx.item()]}")